### Prompts 
- Structured way to create prompts Dynamically by inserting into a predefined template. Instead of hardcoding prompts.
- Reusable, Flexable and easy to manage, especialy when working with Dynamic user input or automation workflows.


- Input to the Model: A prompt is technically the text input that you feed into an LLM to get a specific output. It is the "command" or "query" that triggers the model's generation.
- Instruction & Context: Deep down, a prompt is more than just a question. It usually consists of a few key parts:
    - Instruction: What you want the model to do (e.g., "Summarize this," "Translate this").
    - Context: Background information that helps the model answer    correctly.
    - Input Data: The specific information you want the model to process.
    - Output Indicator: How you want the answer formatted (e.g., "Answer in a list," "Return JSON").
- A "Program" for the LLM: In frameworks like LangChain, prompts are treated like code. Instead of hardcoding a string like "Tell me a joke about cats", we create Prompt Templates (e.g., "Tell me a joke about {animal}"). This allows us to reuse the same structure for different inputs dynamically.
- Steering Mechanism: Prompts are the primary way to "steer" or control the probability distribution of the model's output. A better prompt leads to a more accurate and relevant response (this is the essence of Prompt Engineering).

In [ ]:
# Prompt Template
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from dotenv import load_dotenv
load_dotenv()

llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-7B-Instruct",
    task="text-generation",
    temperature=0.3)
model = ChatHuggingFace(llm=llm)

prompt = PromptTemplate(
    template="Generate few points on {topic}.",
    input_variables=['topic'])

chain = prompt | model
result = chain.invoke({'topic':'Ai'})
print(result.content)


### Theory: Basic Prompt Templates and LCEL

1. **HuggingFaceEndpoint**: This class serves as a connector to models hosted on the Hugging Face Hub (via their Inference API). By specifying `repo_id`, `task`, and `temperature`, we configure which remote model to use and how it should behave (e.g., creativity vs. determinism).

2. **ChatHuggingFace**: While `HuggingFaceEndpoint` provides a raw text completion interface, `ChatHuggingFace` wraps this to provide a chat-oriented interface (handling messages like System, User, AI) which is more compatible with modern instruction-tuned models.

3. **PromptTemplate**: This is a core LangChain concept. Instead of hardcoding prompt strings (e.g., "Tell me about AI"), we create a template with placeholders (e.g., `{topic}`). This allows for:
   - **Reusability**: The same structure can be used for different inputs.
   - **Validation**: LangChain checks if input variables match the template.

4. **LCEL (LangChain Expression Language)**:
   - The syntax `chain = prompt | model` uses the Unix pipe operator `|` to chain components together.
   - Data flows from left to right: The input dictionary `{'topic': 'Ai'}` goes into `prompt`, which formats the string, and the output string goes into `model`, which generates the response.

In [2]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from dotenv import load_dotenv
load_dotenv()

llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-7B-Instruct",
    task="text-generation",
    temperature=0.3)
model = ChatHuggingFace(llm=llm)

prompt = ChatPromptTemplate([
    ('system', '''You are a Helpful AI assistent. your role is to give a short and straight-to-point, direct, concise responces.
     Format: give your answer and then stop...'''),
    MessagesPlaceholder(variable_name='memory'),
    ('human','{query}')])

memory=[]

while True:
    query = input('-->You: ')
    if query.lower() =='exit':        
        break
    chain = prompt | model
    responce = chain.invoke({'query': query,'memory':memory})
    print('-->AI:',responce.content)
    memory.append(HumanMessage(content=query))
    memory.append(AIMessage(content=responce.content))
    

-->AI: Hello!


### Theory: Chat Templates and Memory Management

1. **ChatPromptTemplate**: Unlike a simple string template, this models a conversation history. It accepts a list of role-based messages:
   - `('system', ...)`: Sets the behavior or persona of the AI (e.g., "You are a Helpful AI assistant").
   - `MessagesPlaceholder`: A placeholder for inserting a list of messages dynamically (used here for `memory`).
   - `('human', '{query}')`: The user's current input.

2. **Message Roles**:
   - **HumanMessage**: Represents input from the user.
   - **AIMessage**: Represents the response from the model.
   - **SystemMessage**: (implicit in the tuple) High-level instructions that persist throughout the interaction.

3. **Memory Loop**:
   - The `while True` loop creates a continuous chat session.
   - `memory.append(...)`: We manually store the conversation history.
   - When `chain.invoke` is called, the current `query` AND the entire `memory` list are passed to the prompt template. This allows the model to "remember" previous context (like your name or previous questions) to generate a context-aware response.

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

def messages_to_text(messages):
    lines = []
    for msg in messages:
        if isinstance(msg, HumanMessage):
            lines.append(f"Human: {msg.content}")
        elif isinstance(msg, AIMessage):
            lines.append(f"AI: {msg.content}")
    return "\n".join(lines)


### Theory: Message Serialization

1. **Message Objects**: In LangChain, chat history is stored as objects (`HumanMessage`, `AIMessage`, etc.) rather than simple strings. This preserves metadata and roles.

2. **Serialization**: The function `messages_to_text` converts these objects back into a plain string format. This is often necessary for:
   - **Logging/Debugging**: Printing the conversation history in a readable format.
   - **Storage**: Saving history to a text file or database that doesn't support complex objects.
   - **Model Compatibility**: Some older or raw completion models expect a single string blob rather than a structured list of messages.